In [2]:
!pip install catboost

In [46]:
!pip install text_preprocessing

In [127]:
import os

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score

import numpy as np

from nltk.corpus import stopwords

from catboost import Pool, CatBoostClassifier

from text_preprocessing import to_lower, remove_number, remove_whitespace, preprocess_text, remove_stopword, remove_punctuation, remove_special_character

In [176]:
input_file = '/home/dasha/Documents/ml_crash_course/data/labeled_data_corpus.csv'
if not os.path.exists(input_file):
    raise RuntimeError(f'No input file: {input_file}')

df = pd.read_csv(input_file)
train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print('num rows for train: %d', train_df.shape[0])


num rows for train: %d 5233


In [177]:
X_train = train_df['msg']
y_train = train_df['label']

X_test = test_df['msg']
y_test = test_df['label']

In [178]:
X_train[:10]

0    здравствуйте. ишу 2х спальную квартиру в лимас...
1    #сниму  комнату в лимассоле или недалеко от не...
2                        мошенник риэлторским услугам.
3    **sales**    reg.1053 lic.489/e **stylish apar...
4    важно: [valerii korol](tg://user?id=193474890)...
5    аренда  no: 367/e ️ларнака️между пила и декели...
6    привет  ищу виллу посуточно с бюджетом 2000€ в...
7    важно: [liss](tg://user?id=202814885), если ты...
8                               total messages: 126772
9    аренда  ️ларнака ️в центре города️ saint lazar...
Name: msg, dtype: object

In [179]:
# Preprocess text using custom preprocess functions in the pipeline 
X_train = X_train.str.lower()
X_train = X_train.dropna()

preprocess_functions = [to_lower, remove_punctuation, remove_special_character, remove_number, remove_whitespace]
X_train = X_train.apply(lambda x: preprocess_text(x, preprocess_functions))

X_train = X_train.map(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore'))

#english_stopwords = stopwords.words("english")
#X_train = X_train.apply(lambda x: remove_stopword(x, english_stopwords))

russian_stopwords = stopwords.words("russian")
X_train = X_train.apply(lambda x: remove_stopword(x, russian_stopwords))

greek_stopwords = stopwords.words("greek")
X_train = X_train.apply(lambda x: remove_stopword(x, greek_stopwords))

turkish_stopwords = stopwords.words("turkish")
X_train = X_train.apply(lambda x: remove_stopword(x, turkish_stopwords))

X_train = X_train.str.join(' ')

print(X_train[:10])


0    здравствуйте ишу х спальную квартиру лимассоле...
1    сниму комнату лимассоле недалеко начала август...
2                         мошенник риэлторским услугам
3    sales reg lice stylish apartment with sea view...
4    важно valerii koroltguserid бот спамер пройди ...
5    аренда no e ️ларнака️между пила декелия ️ пеше...
6    привет ищу виллу посуточно бюджетом € сутки дн...
7    важно lisstguserid бот спамер пройди проверку ...
8                                       total messages
9    аренда ️ларнака ️в центре города️ saint lazaro...
Name: msg, dtype: object


In [180]:
unique_words = list(X_train.str.split(' ', expand=True).stack().unique())
len(unique_words)

15580

In [181]:
a = X_train.apply(lambda x: len(x.split(" ")))

In [182]:
a.sum() / len(a)

35.82094400917256

In [183]:
# Preprocess text using custom preprocess functions in the pipeline 
X_test = X_test.str.lower()
X_test = X_test.dropna()

preprocess_functions = [to_lower, remove_punctuation, remove_special_character, remove_number, remove_whitespace]
X_test = X_test.apply(lambda x: preprocess_text(x, preprocess_functions))

X_test = X_test.map(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore'))

#english_stopwords = stopwords.words("english")
#X_test = X_test.apply(lambda x: remove_stopword(x, english_stopwords))

russian_stopwords = stopwords.words("russian")
X_test = X_test.apply(lambda x: remove_stopword(x, russian_stopwords))

greek_stopwords = stopwords.words("greek")
X_test = X_test.apply(lambda x: remove_stopword(x, greek_stopwords))

turkish_stopwords = stopwords.words("turkish")
X_test = X_test.apply(lambda x: remove_stopword(x, turkish_stopwords))

X_test = X_test.str.join(' ')

print(X_test[:10])

5233    сдам собственник лимассол сдаю комнатную новую...
5234    важно tianatguserid бот спамер пройди проверку...
5235    привет аленаtguserid это бесплатная группа аре...
5236    привет meds mtguserid это бесплатная группа ар...
5237    аренда район линопетра лимассол долгосрочную а...
5238                                     клиент приезжает
5239                       james michaeltguserid забанили
5240    сдается квартира пафосе район хлорака м спальн...
5241    аренда х комнатного дома джакузи выходом крышу...
5242    квартира спальнями ванной комнатой верандами к...
Name: msg, dtype: object


In [184]:
X_train = X_train.values

X_test = X_test.values

In [185]:
len(X_train)

5233

In [186]:
X_train[0]

'здравствуйте ишу х спальную квартиру лимассоле желательно гермасойя семья х взрослых х детей животных длительный срок бюджет евро предложения лс'

In [187]:
vectorizer = TfidfVectorizer(max_df=0.3, min_df=0.01).fit(X_train)
X_train_csr = vectorizer.transform(X_train)

In [188]:
X_test_csr = vectorizer.transform(X_test)

In [189]:
X_train_csr

<5233x543 sparse matrix of type '<class 'numpy.float64'>'
	with 100992 stored elements in Compressed Sparse Row format>

In [190]:
train_pool = Pool(
    X_train_csr, 
    y_train
)
valid_pool = Pool(
    X_test_csr, 
    y_test
)

catboost_params = {
    'iterations': 5000,
    'learning_rate': 0.01,
    'eval_metric': 'F1',
    'task_type': 'GPU',
    'early_stopping_rounds': 2000,
    'use_best_model': True,
    'verbose': 500
}
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.7493380	test: 0.7115717	best: 0.7115717 (0)	total: 60.3ms	remaining: 5m 1s
500:	learn: 0.8794821	test: 0.8310139	best: 0.8326693 (454)	total: 24.7s	remaining: 3m 42s
1000:	learn: 0.9160156	test: 0.8410853	best: 0.8410853 (967)	total: 47.9s	remaining: 3m 11s
1500:	learn: 0.9323017	test: 0.8544061	best: 0.8560461 (1482)	total: 1m 10s	remaining: 2m 44s
2000:	learn: 0.9427058	test: 0.8587786	best: 0.8625954 (1930)	total: 1m 33s	remaining: 2m 20s
2500:	learn: 0.9508197	test: 0.8657845	best: 0.8657845 (2460)	total: 1m 57s	remaining: 1m 57s
3000:	learn: 0.9576516	test: 0.8754717	best: 0.8754717 (2982)	total: 2m 19s	remaining: 1m 33s
3500:	learn: 0.9620010	test: 0.8754717	best: 0.8771267 (3040)	total: 2m 43s	remaining: 1m 9s
4000:	learn: 0.9629986	test: 0.8771267	best: 0.8771267 (3040)	total: 3m 5s	remaining: 46.3s
4500:	learn: 0.9654842	test: 0.8750000	best: 0.8771267 (3040)	total: 3m 28s	remaining: 23.1s
4999:	learn: 0.9688249	test: 0.8745247	best: 0.8771267 (3040)	total: 3m 50s	

In [191]:
X_test_csr = vectorizer.transform(X_test)

y_pred = model.predict(X_test_csr)

best_score = f1_score(y_test, y_pred)

In [192]:
print(best_score)

0.8771266540642723


In [193]:
model.save_model("/home/dasha/Documents/ml_crash_course/data/model")
import pickle
pickle.dump(vectorizer, open("/home/dasha/Documents/ml_crash_course/data/vectorizer.pickle", "wb"))

from_file = CatBoostClassifier()

from_file.load_model("/home/dasha/Documents/ml_crash_course/data/model")

loaded_vectorizer = pickle.load(open("/home/dasha/Documents/ml_crash_course/data/vectorizer.pickle", "rb"))

In [195]:
def preprocessing(messages : np.ndarray) -> pd.Series:
    pd_messages = pd.Series(messages)
    pd_messages = pd_messages.str.lower()
    pd_messages = pd_messages.dropna()

    preprocess_functions = [to_lower, remove_punctuation, remove_special_character, remove_number, remove_whitespace]

    pd_messages = pd_messages.apply(lambda x: preprocess_text(x, preprocess_functions))

    pd_messages = pd_messages.map(lambda x: bytes(x, 'utf-8').decode('utf-8', 'ignore'))

    english_stopwords = stopwords.words("english")
    pd_messages = pd_messages.apply(lambda x: remove_stopword(x, english_stopwords))

    russian_stopwords = stopwords.words("russian")
    pd_messages = pd_messages.apply(lambda x: remove_stopword(x, russian_stopwords))

    greek_stopwords = stopwords.words("greek")
    pd_messages = pd_messages.apply(lambda x: remove_stopword(x, greek_stopwords))

    turkish_stopwords = stopwords.words("turkish")
    pd_messages = pd_messages.apply(lambda x: remove_stopword(x, turkish_stopwords))

    pd_messages = pd_messages.str.join(' ')

    return pd_messages

In [196]:
msg = '#аренда #квартира  #лимассол #агент.  ппрп: квартира / 1 спальня    абобус -в конце макариос авеню - апостолос андреас  трехэтажный жилой дом, расположенный в тихом, спокойном жилом районе лимассола, в непосредственной близости от центра города, с легким доступом к близлежащим школам, супермаркетам, а также многим другим услугам , новый порт , молл. квартира с 1 спальней  - открытого плана зал совмещен с кухней  душевая комната  и просторная веранда  крытая парковка и кладовая   все здание теплоизолированное с использованием энергосберегающих технологий класса а, а все окна оснащены двойным тепловым остеклением. квартира сдается  без мебели, будет  оборудована электроприборами и установлены кондиционеры. идеальное тихое место в центре города! цена снижена : €1,050 /per month (plus 1 deposit )'

pre_proc = preprocessing(np.array( [msg] ))

X_test_csr = loaded_vectorizer.transform(pre_proc)


pred = from_file.predict(X_test_csr)

print(pred[0])


1
